In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

# Completion model

In [2]:
from langchain_openai import OpenAI
llmModel = OpenAI()

# Chat completion model

In [7]:
from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model="gpt-4.1-mini")

# Prompts and Prompt Templates

In [ ]:
# This is for completions model
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} sory about {topic}"
)

llmModelPrompt = prompt_template.format(
    adjective="curious",
    topic="The Kennedy family"
)

res = llmModel.invoke(llmModelPrompt)
print(res)

In [9]:
# This is for chat completions model
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an {profession} expert on {topic}."),
        ("human", "Hello, Mr. {profession}, can you please answer a question?"),
        ("ai", "Sure!"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(
    profession="Historian",
    topic="The Kennedy family",
    user_input="How many grandchildren Joseph P. Kennedy had?"
)

response = chatModel.invoke(messages)
print(response.content)

Joseph P. Kennedy Sr. and Rose Fitzgerald Kennedy had nine children, many of whom had children of their own, resulting in a large number of grandchildren.

Joseph P. Kennedy Sr. had 29 grandchildren in total. If you'd like, I can provide more details about some of his notable grandchildren or the Kennedy family's subsequent generations.


# Old way

In [ ]:
# from langchain_core.messages import SystemMessage
# from langchain_core.prompts import HumanMessagePromptTemplate

# chat_template = ChatPromptTemplate.from_messages(
#     [
#         SystemMessage(
#             content=(
#                 "You are an Historian expert on the Kennedy family."
#             )
#         ),
#         HumanMessagePromptTemplate.from_template("{user_input}"),
#     ]
# )

# messages = chat_template.format_messages(
#     user_input="Name the children and grandchildren of Joseph P. Kennedy?"
# )

# response = chatModel.invoke(messages)

In [ ]:
# response.content

"Joseph P. Kennedy Sr. and his wife Rose Fitzgerald Kennedy had nine children. Here are their names:\n\n1. Joseph P. Kennedy Jr. (1915–1944)\n2. John F. Kennedy (1917–1963)\n3. Rosemary Kennedy (1918–2005)\n4. Kathleen Kennedy (1920–1948)\n5. Eunice Kennedy (1921–2009)\n6. Patricia Kennedy (1924–2006)\n7. Robert F. Kennedy (1925–1968)\n8. Jean Kennedy (born 1928)\n9. Edward M. Kennedy (1932–2009)\n\nAs for grandchildren, the Kennedy family is quite extensive. Here are some notable grandchildren of Joseph P. Kennedy Sr.:\n\n- Children of John F. Kennedy and Jacqueline Kennedy Onassis:\n  - Caroline Kennedy (born 1957)\n  - John F. Kennedy Jr. (1960–1999)\n\n- Children of Robert F. Kennedy and Ethel Kennedy:\n  - Kathleen Kennedy Townsend (born 1951)\n  - Joseph P. Kennedy II (born 1952)\n  - Robert F. Kennedy Jr. (born 1954)\n  - David Kennedy (1955–1984)\n  - Mary Kerry Kennedy (born 1959)\n  - Christopher G. Kennedy (born 1963)\n  - Max Kennedy (born 1965)\n  - Douglas Kennedy (born 1

# Few shot prompting

In [ ]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

examples = [
    {"input": "hi!", "output": "¡hola!"},
    {"input": "bye!", "output": "¡adiós!"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an English-Spanish translator."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# Chain

In [13]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

examples = [
    {"input": "hi!", "output": "¡hola!"},
    {"input": "bye!", "output": "¡adiós!"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an English-Spanish translator."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

chain = final_prompt | chatModel

res = chain.invoke({"input": "How are you?"})
print(res.content)

¿Cómo estás?


# Langchain Expressions Language

# Output Parsers

In [14]:
from langchain_openai import OpenAI
llmModel = OpenAI()

In [ ]:
from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model="gpt-4.1-mini")

In [ ]:
# Completion
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers.json import SimpleJsonOutputParser

json_prompt = PromptTemplate.from_template(
    "Return a JSON object with an `answer` key that answers the following question: {question}"
)

json_parser = SimpleJsonOutputParser()

json_chain = json_prompt | llmModel | json_parser

In [ ]:
res = json_chain.invoke({"question": "What is the biggest contry?"})

# Optionally, you can use Pydantic to define a custom output format

In [17]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

In [18]:
# Define a Pydantic Object with the desired output format.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

In [19]:
# Define the parser referring the Pydantic Object
parser = JsonOutputParser(pydantic_object=Joke)

# Add the parser format instructions in the prompt definition.
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Create a chain with the prompt and the parser
chain = prompt | chatModel | parser

chain.invoke({"query": "Tell me a joke."})

{'setup': "Why don't scientists trust atoms?",
 'punchline': 'Because they make up everything!'}